In [1]:
import boto3

In [2]:
%%writefile ../guide_scripts/list_foundation_models.py
import logging
import json
import boto3

# Lists the available Amazon Bedrock models.

from botocore.exceptions import ClientError

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.
    :return: The list of available bedrock foundation models.
    """
    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models
    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise Exception("Couldn't list foundation models.")


def main():
    """Entry point for the example. Uses the AWS SDK for Python (Boto3)
    to create an Amazon Bedrock client. Then lists the available Bedrock models
    in the region set in the callers profile and credentials.
    """
    bedrock_client = boto3.client(service_name="bedrock")
    fm_models = list_foundation_models(bedrock_client)
    for model in fm_models:
        print(f"Model: {model['modelName']}")
        print(json.dumps(model, indent=2))
        print("---------------------------\n")
    logger.info("Done.")

if __name__ == "__main__":
    main()

Overwriting ../guide_scripts/list_foundation_models.py


In [3]:
%%writefile ../guide_scripts/invoke_titan_text_express.py
# Use the native inference API to send a text message to Amazon Titan Text G1 -
# Express.
import boto3
import json
from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")
# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"
# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."
# Format the request payload using the model's native structure.
native_request = {
    "inputText": prompt,
    "textGenerationConfig": {
    "maxTokenCount": 512,
    "temperature": 0.7,
    "topP": 0.9
    },
}
# Convert the native request to JSON.
request = json.dumps(native_request)
try:
    # Invoke the model with the request.
    response = brt.invoke_model(modelId=model_id, body=request)
except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())
# Extract and print the response text.
response_text = model_response["results"][0]["outputText"]

print(f"Response: {response_text}")

Overwriting ../guide_scripts/invoke_titan_text_express.py


In [4]:
%%writefile ../guide_scripts/generate_text_response_converse.py
import boto3
from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

inference_config = {
     'maxTokens': 512,
     'temperature': 0.5,
     'topP': 0.9
}
performanceConfig={
    'latency': 'standard'
}
# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    },
    {
        "role": "assistant",
        "content": [{"text": "A 'hello world' program demonstrates the basic syntax of a programming language."}],
    },
    {
        "role": "user",
        "content": [{"text": "What is the purpose of a 'hello world' program in Python?"}],
    },
]

try:
    response = brt.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig=inference_config,
        performanceConfig=performanceConfig,
    )
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)
except ClientError as e:
    print(f"An error occurred: {e}")

Overwriting ../guide_scripts/generate_text_response_converse.py


In [5]:
aws_region = "us-east-1"
bedrock_client = boto3.client(service_name="bedrock", region_name=aws_region)

### Using DSPY and Bedrock CoT (Proof of Concept)


In [6]:
import os
from aws_keys import AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_REGION

# Set the AWS credentials as environment variables
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_REGION"] = AWS_REGION

In [7]:
import dspy
lm = dspy.LM('bedrock/amazon.nova-pro-v1:0')
dspy.configure(lm=lm)

In [8]:
qa = dspy.Predict("question:str -> answer")

In [10]:
cot = dspy.ChainOfThought("question -> answer")
question = "True of False: The numbers in this group add up to an even number: 17,  8, 10, 12, 13, 4, 2."
cot(question=question).answer

'True'

### Continuing Bedrock Samples

In [11]:
%%writefile ../guide_scripts/meeting_transcript.py
import json
import logging
import boto3

from botocore.exceptions import ClientError


"""
Shows how to create a list of action items from a meeting transcript
with the Amazon Titan Text model (on demand).
"""


class ImageError(Exception):
    "Custom exception for errors returned by Amazon Titan Text models"

    def __init__(self, message):
        self.message = message


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def generate_text(model_id, body):
    """
    Generate text using Amazon Titan Text models on demand.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        response (json): The response from the model.
    """

    logger.info(
        "Generating text with Amazon Titan Text model %s", model_id)

    bedrock = boto3.client(service_name='bedrock-runtime')

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type
    )
    response_body = json.loads(response.get("body").read())

    finish_reason = response_body.get("error")

    if finish_reason is not None:
        raise ImageError(f"Text generation error. Error is {finish_reason}")

    logger.info(
        "Successfully generated text with Amazon Titan Text model %s", model_id)

    return response_body


def main():
    """
    Entrypoint for Amazon Titan Text model example.
    """
    try:
        logging.basicConfig(level=logging.INFO,
                            format="%(levelname)s: %(message)s")

        # You can replace the model_id with any other Titan Text Models
        # Titan Text Model family model_id is as mentioned below:
        # amazon.titan-text-premier-v1:0, amazon.titan-text-express-v1, amazon.titan-text-lite-v1
        model_id = 'amazon.titan-text-express-v1'

        prompt = """Meeting transcript: Miguel: Hi Brant, I want to discuss the workstream  
            for our new product launch Brant: Sure Miguel, is there anything in particular you want
            to discuss? Miguel: Yes, I want to talk about how users enter into the product.
            Brant: Ok, in that case let me add in Namita. Namita: Hey everyone 
            Brant: Hi Namita, Miguel wants to discuss how users enter into the product.
            Miguel: its too complicated and we should remove friction.  
            for example, why do I need to fill out additional forms?  
            I also find it difficult to find where to access the product
            when I first land on the landing page. Brant: I would also add that
            I think there are too many steps. Namita: Ok, I can work on the
            landing page to make the product more discoverable but brant
            can you work on the additonal forms? Brant: Yes but I would need 
            to work with James from another team as he needs to unblock the sign up workflow.
            Miguel can you document any other concerns so that I can discuss with James only once?
            Miguel: Sure.
            From the meeting transcript above, Create a list of action items for each person. """

        body = json.dumps({
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 4096,
                "stopSequences": [],
                "temperature": 0.7,
                "topP": 0.9
            }
        })

        response_body = generate_text(model_id, body)
        print(f"Input token count: {response_body['inputTextTokenCount']}")

        for result in response_body['results']:
            print(f"Token count: {result['tokenCount']}")
            print(f"Output text: {result['outputText']}")
            print(f"Completion reason: {result['completionReason']}")

    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
              format(message))
    except ImageError as err:
        logger.error(err.message)
        print(err.message)

    else:
        print(
            f"Finished generating text with the Amazon Titan Text Premier model {model_id}.")


if __name__ == "__main__":
    main()

Overwriting ../guide_scripts/meeting_transcript.py


In [ ]:
# %%writefile ../guide_scripts/tool_call.py
import logging
import json
import boto3


from botocore.exceptions import ClientError

"""
Shows how to use tools with the <noloc>Converse</noloc> API and the Cohere Command R model.
"""


class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""

    pass


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def get_top_song(call_sign):
    """Returns the most popular song for the requested station.
    Args:
        call_sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    if call_sign == "WZPZ":
        song = "Elemental Hotel"
        artist = "8 Storey Hike"

    else:
        raise StationNotFoundError(f"Station {call_sign} not found.")

    return song, artist


def generate_text(bedrock_client, model_id, tool_config, input_text):
    """Generates text using the supplied Amazon Bedrock model. If necessary,
    the function handles tool use requests and sends the result to the model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The Amazon Bedrock model ID.
        tool_config (dict): The tool configuration.
        input_text (str): The input text.
    Returns:
        Nothing.
    """

    logger.info("Generating text with model %s", model_id)

    # Create the initial message from the user input.
    messages = [{"role": "user", "content": [{"text": input_text}]}]

    response = bedrock_client.converse(
        modelId=model_id, messages=messages, toolConfig=tool_config
    )

    output_message = response["output"]["message"]
    messages.append(output_message)
    stop_reason = response["stopReason"]

    if stop_reason == "tool_use":
        # Tool use requested. Call the tool and send the result to the model.
        tool_requests = response["output"]["message"]["content"]
        for tool_request in tool_requests:
            if "toolUse" in tool_request:
                tool = tool_request["toolUse"]
                logger.info(
                    "Requesting tool %s. Request: %s", tool["name"], tool["toolUseId"]
                )

                if tool["name"] == "top_song":
                    tool_result = {}
                    try:
                        song, artist = get_top_song(tool["input"]["sign"])
                        tool_result = {
                            "toolUseId": tool["toolUseId"],
                            "content": [{"json": {"song": song, "artist": artist}}],
                        }
                    except StationNotFoundError as err:
                        tool_result = {
                            "toolUseId": tool["toolUseId"],
                            "content": [{"text": err.args[0]}],
                            "status": "error",
                        }

                    tool_result_message = {
                        "role": "user",
                        "content": [{"toolResult": tool_result}],
                    }
                    messages.append(tool_result_message)

                    # Send the tool result to the model.
                    response = bedrock_client.converse(
                        modelId=model_id, messages=messages, toolConfig=tool_config
                    )
                    output_message = response["output"]["message"]

    # print the final response from the model.
    for content in output_message["content"]:
        print(json.dumps(content, indent=4))


def main():
    """
    Entrypoint for tool use example.
    """

    logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

    model_id = "amazon.nova-pro-v1:0"
    input_text = "What is the most popular song on WZPZ?"

    tool_config = {
        "tools": [
            {
                "toolSpec": {
                    "name": "top_song",
                    "description": "Get the most popular song played on a radio station.",
                    "inputSchema": {
                        "json": {
                            "type": "object",
                            "properties": {
                                "sign": {
                                    "type": "string",
                                    "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ, and WKRP.",
                                }
                            },
                            "required": ["sign"],
                        }
                    },
                }
            }
        ]
    }
    bedrock_client = boto3.client(service_name="bedrock-runtime")

    try:
        print(f"Question: {input_text}")
        generate_text(bedrock_client, model_id, tool_config, input_text)

    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")

    else:
        print(f"Finished generating text with model {model_id}.")


if __name__ == "__main__":
    main()


INFO:__main__:Generating text with model amazon.nova-pro-v1:0


Question: What is the most popular song on WZPZ?


INFO:__main__:Requesting tool top_song. Request: tooluse_eVs-vdanS365uOINnTtaHQ


{
    "text": "The most popular song on WZPZ is \"Elemental Hotel\" by 8 Storey Hike."
}
Finished generating text with model amazon.nova-pro-v1:0.


## Bedrock Agent Prompt Management

In [20]:
# Create a prompt in Prompt management
import boto3

# Create an Amazon Bedrock Agents client
client = boto3.client(service_name="bedrock-agent")

# Create the prompt
response = client.create_prompt(
    name="MakePlaylist1",
    description="My first prompt.",
    variants=[
        { 
            "name": "Variant1",
            "modelId": "amazon.titan-text-express-v1",
            "templateType": "TEXT",
            "inferenceConfiguration": {
                "text": {
                    "temperature": 0.8
                }
            },
            "templateConfiguration": { 
                "text": {
                    "text": "Make me a {{genre}} playlist consisting of the following number of songs: {{number}}."
                }
            }
      }
    ]
)
                        
prompt_id = response.get("id")

In [22]:
client.list_prompts()
client.get_prompt(promptIdentifier=prompt_id)
# Create a version of the prompt that you created
response = client.create_prompt_version(promptIdentifier=prompt_id)
                        
prompt_version = response.get("version")
prompt_version_arn = response.get("arn")
print(f"Prompt version ARN: {prompt_version_arn}, Prompt version: {prompt_version}")


Prompt version ARN: arn:aws:bedrock:us-east-1:914839022649:prompt/VI56XO1NBZ:2, Prompt version: 2


In [26]:
client.list_prompts(promptIdentifier='VI56XO1NBZ')
client.get_prompt(promptIdentifier='VI56XO1NBZ', promptVersion=prompt_version)

{'ResponseMetadata': {'RequestId': '30e024cb-7111-4071-bc4e-8dbe4b80176a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 19 Apr 2025 15:55:22 GMT',
   'content-type': 'application/json',
   'content-length': '507',
   'connection': 'keep-alive',
   'x-amzn-requestid': '30e024cb-7111-4071-bc4e-8dbe4b80176a',
   'x-amz-apigw-id': 'JRwYrHicIAMEkeA=',
   'x-amzn-trace-id': 'Root=1-6803c76a-03047257691a074f2cab7e98'},
  'RetryAttempts': 0},
 'arn': 'arn:aws:bedrock:us-east-1:914839022649:prompt/VI56XO1NBZ:2',
 'createdAt': datetime.datetime(2025, 4, 19, 15, 53, 30, 159112, tzinfo=tzutc()),
 'id': 'VI56XO1NBZ',
 'name': 'MakePlaylist1',
 'updatedAt': datetime.datetime(2025, 4, 19, 15, 53, 30, 159112, tzinfo=tzutc()),
 'variants': [{'inferenceConfiguration': {'text': {'temperature': 0.800000011920929}},
   'modelId': 'amazon.titan-text-express-v1',
   'name': 'Variant1',
   'templateConfiguration': {'text': {'text': 'Make me a {{genre}} playlist consisting of the following number o

In [31]:
%%writefile ../guide_scripts/prompt_management_flow.py
# Import Python SDK and create client
import boto3

client = boto3.client(service_name='bedrock-agent')

FLOWS_SERVICE_ROLE = "arn:aws:iam::123456789012:role/MyPromptFlowsRole" # Flows service role that you created. For more information, see https://docs.aws.amazon.com/bedrock/latest/userguide/flows-permissions.html
PROMPT_ARN = prompt_version_arn # ARN of the prompt that you created, retrieved programatically during creation.

# Define each node

# The input node validates that the content of the InvokeFlow request is a JSON object.
input_node = {
    "type": "Input",
    "name": "FlowInput",
    "outputs": [
        {
            "name": "document",
            "type": "Object"
        }
    ]
}

# This prompt node contains a prompt that you defined in Prompt management.
# It validates that the input is a JSON object that minimally contains the fields "genre" and "number", which it will map to the prompt variables.
# The output must be named "modelCompletion" and be of the type "String".
prompt_node = {
    "type": "Prompt",
    "name": "MakePlaylist",
    "configuration": {
        "prompt": {
            "sourceConfiguration": {
                "resource": {
                    "promptArn": ""
                }
            }
        }
    },
    "inputs": [
        {
            "name": "genre",
            "type": "String",
            "expression": "$.data.genre"
        },
        {
            "name": "number",
            "type": "Number",
            "expression": "$.data.number"
        }
    ],
    "outputs": [
        {
            "name": "modelCompletion",
            "type": "String"
        }
    ]
}

# The output node validates that the output from the last node is a string and returns it as is. The name must be "document".
output_node = {
    "type": "Output",
    "name": "FlowOutput",
    "inputs": [
        {
            "name": "document",
            "type": "String",
            "expression": "$.data"
        }
    ]
}

# Create connections between the nodes
connections = []

#   First, create connections between the output of the flow input node and each input of the prompt node
for input in prompt_node["inputs"]:
    connections.append(
        {
            "name": "_".join([input_node["name"], prompt_node["name"], input["name"]]),
            "source": input_node["name"],
            "target": prompt_node["name"],
            "type": "Data",
            "configuration": {
                "data": {
                    "sourceOutput": input_node["outputs"][0]["name"],
                    "targetInput": input["name"]
                }
            }
        }
    )

# Then, create a connection between the output of the prompt node and the input of the flow output node
connections.append(
    {
        "name": "_".join([prompt_node["name"], output_node["name"]]),
        "source": prompt_node["name"],
        "target": output_node["name"],
        "type": "Data",
        "configuration": {
            "data": {
                "sourceOutput": prompt_node["outputs"][0]["name"],
                "targetInput": output_node["inputs"][0]["name"]
            }
        }
    }
)

# Create the flow from the nodes and connections
client.create_flow(
    name="FlowCreatePlaylist",
    description="A flow that creates a playlist given a genre and number of songs to include in the playlist.",
    executionRoleArn=FLOWS_SERVICE_ROLE,
    definition={
        "nodes": [input_node, prompt_node, output_node],
        "connections": connections
    }
)

Overwriting ../guide_scripts/prompt_management_flow.py
